In [30]:
from nltk.probability import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, ELEProbDist

In [31]:
def sentiment_feature(word):
	return {'word': word}


In [3]:
import random
import nltk
from nltk.corpus import opinion_lexicon
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

text = opinion_lexicon

positive_words = ([(lemmatizer.lemmatize(word=word), 1.0) for word in text.positive()])
negative_words = ([(lemmatizer.lemmatize(word=word), 0.0) for word in text.negative()])

labeled_words = positive_words + negative_words
random.shuffle(labeled_words)

featuresets = [(sentiment_feature(n[0]), n[1]) for n in labeled_words]

train_amnt = int(len(featuresets)*.8) #80% train data; 20% test data
train_data = featuresets[:train_amnt]
test_data = featuresets[train_amnt:]
assert len(train_data) == train_amnt

In [4]:
from nltk.probability import ConditionalFreqDist, ConditionalProbDist
cfdist = ConditionalFreqDist(labeled_words)
cpdist = ConditionalProbDist(cfdist, ELEProbDist, bins=2)

word = 'separate'

print("word: ", word)
#print("Number of occurances in opinion_lexicon: ", cpdist[word].max())
print("sentiment: ", (cpdist[word].prob(1.0) * 1))
print("probability of recieving a sentiment of 1.0: ", (cpdist[word].prob(1.0) * 1))
print("probability of recieving a sentiment of 0.0: ", (cpdist[word].prob(0.0) * 1))

word:  separate
sentiment:  0.5
probability of recieving a sentiment of 1.0:  0.5
probability of recieving a sentiment of 0.0:  0.5


In [5]:
from nltk.probability import LaplaceProbDist, LidstoneProbDist


In [6]:
from nltk.classify import NaiveBayesClassifier
from nltk.probability import LaplaceProbDist
from nltk.probability import ConditionalFreqDist


from nltk.probability import ProbDistI, FreqDist


freq_dist = {}

#freqDist = FreqDist()
#probDist = ProbDistI(freqDist)

#dict value = [occurances of -1, occurances of 1]
for wrd in labeled_words:
    
    default = 0
    
    wrd_tmp = str(wrd[0])
    val = int(wrd[1])
    
    freq_dist[wrd_tmp] = [0,0]
    
    if(val == 1):
        freq_dist[wrd_tmp][1] += 1
    elif(val == -1):
        freq_dist[wrd_tmp][0] += 1 
    
#LaplaceProbDist()



#classifier = nltk.NaiveBayesClassifier.train()

In [7]:
import os, re, nltk
from Models.LoadAndTrainModels import sentiment_feature_sentence, applyFeatureSetToBigrams, sentiment_feature

data_path = os.path.join('/home/brf97486/finalProject/labelled_sentence_data_set/sentiment labelled sentences')
yelp = os.path.join(data_path, 'yelp_labelled.txt')
imdb = os.path.join(data_path, 'imdb_labelled.txt')
amazon = os.path.join(data_path, 'amazon_cells_labelled.txt')
assert os.path.isfile(yelp) 
assert os.path.isfile(imdb) 
assert os.path.isfile(amazon) 
file = open(yelp, 'r')
file2 = open(imdb, 'r')
file3 = open(amazon, 'r')
dataset = []
for line in file:
    dataset.append(re.split(r'\t+', line))
for line in file2:
    dataset.append(re.split(r'\t+', line))
for line in file3:
    dataset.append(re.split(r'\t+', line))

In [8]:
ds = []
#nltk.bigram(dataset)
for line in dataset:
    value = int(line[1])
    words = nltk.word_tokenize(line[0])
    bigram = list(nltk.bigrams(words))
    #bigram_with_features = applyFeatureSetToBigrams(bigram)
    for bi in bigram:
       ds.append((value, ' '.join(bi)))

In [9]:
cfdist = ConditionalFreqDist(ds)

In [10]:
bigram_features = list(cfdist.keys())
print(bigram_features)

[1, 0]


In [11]:
featuresets = [print(bi) for bi in bigram]

('You', 'can')
('can', 'not')
('not', 'answer')
('answer', 'calls')
('calls', 'with')
('with', 'the')
('the', 'unit')
('unit', ',')
(',', 'never')
('never', 'worked')
('worked', 'once')
('once', '!')


In [12]:
from getExternalDatasets import *
train, test, prob = getUCIDataset_bigram()

In [13]:
cfdist[('not good')]

FreqDist({})

In [19]:
from nltk.probability import DictionaryProbDist
    
dict_probs = {0.0: .1, 1.0 : .1}
label_probdist = DictionaryProbDist(dict_probs)

classifier = nltk.NaiveBayesClassifier(label_probdist=label_probdist, feature_probdist=cfdist)

In [20]:
print("Classifier accuracy percent: ", (nltk.classify.accuracy(classifier, test))*100)

Classifier accuracy percent:  47.17868338557994


In [22]:
prob_classify = classifier.prob_classify({'bigram': 'very good'})
print(prob_classify._prob_dict)

{0.0: -1.0, 1.0: -1.0}


In [17]:
classifier.show_most_informative_features()

Most Informative Features


TypeError: 'int' object is not iterable

In [28]:

from Models import LoadAndTrainModels
from Make_Enron_Corpus import enronCorpus
from analyze_data.Data_Utils import *
from getExternalDatasets import *

In [29]:
word_set = LoadAndTrainModels.applyFeatureSetToWords(enronCorpus().words())


#training_data, testing_data, prob_dist = LoadAndTrainModels.getOpinionTrainingDataWords()

#get the trained sentiment classifier
classifier = LoadAndTrainModels.getNaiveBayesTrainedClassifier(LoadAndTrainModels.getOpinionTrainingDataWordsAndProbDist)



#classify each relationship type and store the average in a dict
avg_sentiment = {}

for d in enron_corp.fileids():
    break
    relation_data = enron_corp.words(d) #data for a single relation
    relation_feature_data = LoadAndTrainModels.applyFeatureSetToWords(relation_data) #data for a single relation with a feature set application to it
    print(type(relation_feature_data))
    total_sentiment = 0
    total_sentiment = classifier.classify_many(featuresets=relation_feature_data)
    avg_sentiment[d] = sum(total_sentiment)/len(relation_feature_data)

print("average sentiment for each relation type: ", avg_sentiment)

print("classifying: envious")
prob_classify = (classifier.prob_classify({"word":"envious"}))
print(prob_classify._prob_dict)

print("classifying: good")
prob_classify = (classifier.prob_classify({"word":"good"}))
print(prob_classify._prob_dict)

print("classifying: bad")
prob_classify = (classifier.prob_classify({"word":"bad"}))
print(prob_classify._prob_dict)

Classifier accuracy percent:  28.424153166421206
average sentiment for each relation type:  {}
classifying: envious
{0.0: -1.0, 1.0: -1.0}
classifying: good
{0.0: -1.0, 1.0: -1.0}
classifying: bad
{0.0: -1.0, 1.0: -1.0}
